In [53]:
## IMPORT MODULES
import pandas as pd
import numpy as np

In [54]:
## IMPORT PLOTTING MODULES
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import matplotlib as mpl
from matplotlib import cm
%matplotlib widget

In [55]:
# TEMPLATE FOR LINEAR REGRESSION
from sklearn.linear_model import LinearRegression

#X = [var1 var2]
#y = var3
def linreg(X ,Y):
    # perform multivariate or univariate
    reg = LinearRegression(normalize=True)
    reg.fit(X,Y)

    intercept = reg.intercept_ 
    coef = reg.coef_
    
    R2 = reg.score(X,Y) # built in function for r^2
    
    ypred = reg.predict(X) # plot x vs. ypred for line or plane
    
    return ypred, intercept,coef,R2

def linreg2(x1,x2,y):
    # function to perfrom both multivariate and univarate regression
    
    # input: 1) df['x1'] -> no need to do anything special, just make sure there are no NaN values
    #        2) df['x2']
    #        3) df['y']
    
    # Output: 1) x1 array -> formatted for plotting (1d np arrary)
    #         2) x2 array
    #         3) y array
    #         4) multivariate results in tuple
    #         5) univariate (x1) results in tuple
    #         6) univariate (x2)results in tuple
    #            -> tuple indices are: 0) ypred (ready for plotting plane or line)
    #                                  1) intercepts
    #                                  2) coefficients
    #                                  3) R2 

    
    length = len(x1)
    x1,x2,Y = x1.values.reshape(length,1),x2.values.reshape(length,1),y.values.reshape(length,1)
    
    # start with multivaraite
    X = np.concatenate((x1,x2),axis = 1)
    reg = LinearRegression(normalize=True)
    reg.fit(X,Y)

    intercept = reg.intercept_ 
    coef = reg.coef_
    
    R2 = reg.score(X,Y) # built in function for r^2
    
    ypred = reg.predict(X) # plot x vs. ypred for line or plane
    
    mv_vals = (ypred.reshape(length,), intercept,coef,R2)
    
    # then do x1 only
    reg = LinearRegression(normalize=True)
    reg.fit(x1,Y)

    intercept = reg.intercept_ 
    coef = reg.coef_
    
    R2 = reg.score(x1,Y) # built in function for r^2
    
    ypred = reg.predict(x1) # plot x vs. ypred for line or plane
    
    x1_vals = (ypred.reshape(length,), intercept,coef,R2)
    
    # then do x2 only
    reg = LinearRegression(normalize=True)
    reg.fit(x2,Y)

    intercept = reg.intercept_ 
    coef = reg.coef_
    
    R2 = reg.score(x2,Y) # built in function for r^2
    
    ypred = reg.predict(x2) # plot x vs. ypred for line or plane
    
    x2_vals = (ypred.reshape(length,), intercept,coef,R2)
    
    x1 = x1.reshape(length,)
    x2 = x2.reshape(length,)
    x1.reshape(length,)
    return x1,x2,Y,mv_vals,x1_vals,x2_vals
    

In [56]:
## FULL DATAFRAME
df_raw = pd.read_csv("Wide-Short_Summary.csv")
df_raw_avg = df_raw.groupby(['vz_d','vx_d']).agg([np.mean,np.std]).reset_index()

df = df_raw
df['My_d'] = df['My_d'].apply(lambda x: np.abs(x))
df = df.drop(df[df['My_d']>=7.7].index)
df = df.dropna()

## AVERAGED DATAFRAME
df_avg = df.groupby(['vz_d','vx_d']).agg([np.mean,np.std]).reset_index()

In [57]:

## LANDING RATE DATA
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")
ax.scatter(df_raw['vx_d'],df_raw['vz_d'],df_raw['landing_rate'])

ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('Landing Rate')
ax.set_title('Landing Rate (Raw Data)')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [58]:
# LINEAR REGRESSION FOR VZ/VX VS LANDING RATE
# tmp_df = df_raw.dropna()
# length = len(tmp_df['vx_d'].values)
# X = np.concatenate((tmp_df['vx_d'].values.reshape(length,1),tmp_df['vz_d'].values.reshape(length,1)),axis=1)
# Y = tmp_df['landing_rate'].values.reshape(length,1)
# ypred,inter,coef,r2 = linreg(X,Y)

tmp_df = df_raw.dropna()
x1,x2,y,mv,var1,var2 = linreg2(tmp_df['vx_d'],tmp_df['vz_d'],tmp_df['landing_rate'])
print('vx_d / vz_d vs landing_rate r2 = ',mv[3])
print('vx_d vs landing_rate r2 = ',var1[3])
print('vz_d vs landing_rate r2 = ',var2[3])

ax.plot_trisurf(x1.T,x2.T,mv[0].T)
plt.show()

# LR FOR VZ VS LR

#LR FOR VZ VS LZ

vx_d / vz_d vs landing_rate r2 =  0.5132987877864901
vx_d vs landing_rate r2 =  0.4784595066849612
vz_d vs landing_rate r2 =  0.02074605856835976


In [59]:

## AVG LANDING RATE SURFACE
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")
cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=0,vmax=1)
pcm = ax.plot_trisurf(df_raw_avg['vx_d'],df_raw_avg['vz_d'],df_raw_avg['landing_rate','mean'],cmap = cmap,norm=norm)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing %")
df
ax.set_zlim(0,1)

ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('Landing Rate')
ax.set_title('Avg Landing Rate for Final 3 Episodes')


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [60]:
## AVG LANDING RATE SURFACE
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

from scipy.interpolate import griddata


X = df_raw_avg['vx_d']
Y = df_raw_avg['vz_d']
Z = df_raw_avg['landing_rate','mean']

xi = np.linspace(X.min(),X.max(),(len(Z)//3))
yi = np.linspace(Y.min(),Y.max(),(len(Z)//3))
zi = griddata((X, Y), Z, (xi[None,:], yi[:,None]), method='linear')
xig, yig = np.meshgrid(xi, yi)

cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=0,vmax=1)


surf = ax.plot_surface(xig, yig, zi,cmap=cmap,norm=norm)
# surf = ax.contour(xig, yig, zi,cmap=cmap,norm=norm)

fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing %")
ax.set_zlim(0,1)

ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('Landing Rate')
ax.set_title('Avg Landing Rate for Final 3 Episodes')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-60-5dd727cb7434>:21: UserWarning: Z contains NaN values. This may result in rendering artifacts.
  surf = ax.plot_surface(xig, yig, zi,cmap=cmap,norm=norm)


In [61]:
# LANDING RATE VS VX AND VY REGRESSION 



In [62]:
## RREV Threshold vs IC
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

ax.scatter(df['vx_d'],df['vz_d'],df['RREV_threshold'])


ax.set_xlim(0,2.75)
ax.set_ylim(1.0,4.5)


ax.set_xlabel('Vx_d')
ax.set_ylabel('Vz_d')
ax.set_zlabel('RREV_threshold')
ax.set_title('RREV_thr vs IC - (Raw Data)')

x1,x2,y,mv,var1,var2 = linreg2(tmp_df['vx_d'],tmp_df['vz_d'],tmp_df['RREV_threshold'])
print('vx_d / vz_d vs RREV r2 = ',mv[3])
print('vx_d vs RREV r2 = ',var1[3])
print('vz_d vs RREV r2 = ',var2[3])

ax.plot_trisurf(x1.T,x2.T,mv[0].T)
plt.show()





Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

vx_d / vz_d vs RREV r2 =  0.5581221162192702
vx_d vs RREV r2 =  0.06530812876702274
vz_d vs RREV r2 =  0.5131024054007607


In [63]:

## Avg RREV_thr vs IC
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=4,vmax=5)
pcm = ax.plot_trisurf(df_avg['vx_d'],df_avg['vz_d'],df_avg['RREV_threshold','mean'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="RREV_threshold")

ax.set_xlim(0,2.75)
ax.set_ylim(1.0,4.5)
ax.set_zlim(3,6)

ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('RREV_threshold')

ax.set_title('Avg RREV_thr vs IC')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [64]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=4,vmax=5)
pcm = ax.scatter(df['vx_d'],df['vz_d'],df['impact_tdelta'])
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="RREV_threshold")


ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('Delta_t [s]')

ax.set_title('Time Rotating vs IC (Raw Data)')

ax.set_zlim(0,0.5)


x1,x2,y,mv,var1,var2 = linreg2(tmp_df['vx_d'],tmp_df['vz_d'],tmp_df['impact_tdelta'])
print('vx_d / vz_d vs impact_tdelta r2 = ',mv[3])
print('vx_d vs impact_tdelta r2 = ',var1[3])
print('vz_d vs impact_tdelta r2 = ',var2[3])

ax.plot_trisurf(x1.T,x2.T,mv[0].T)
plt.show()




Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

vx_d / vz_d vs impact_tdelta r2 =  0.4164235632584987
vx_d vs impact_tdelta r2 =  0.011098727540745035
vz_d vs impact_tdelta r2 =  0.4120346424404435


In [69]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")


cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=4)

pcm = ax.scatter(df['RREV_trigger'],df['landing_rate'],df['impact_tdelta'],c=df['vz_d'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Vz_d')


ax.set_xlabel('RREV_trigger')
ax.set_ylabel('Landing_rate')
ax.set_zlabel('Delta_t [s]')

ax.set_title('Time Rotating vs RREV (Raw Data)')

ax.set_zlim(0,0.5)

x1name = 'RREV_trigger'
x2name = 'landing_rate'
yname = 'impact_tdelta'

x1,x2,y,mv,var1,var2 = linreg2(tmp_df[x1name],tmp_df[x2name],tmp_df[yname])
print(x1name+" / " + x2name + " vs. " + yname+ " r2 = ", mv[3])
print(x1name + " vs. " + yname + " r2 =" , var1[3])
print(x2name + " vs. " + yname + " r2 =" , var2[3])

ax.plot_trisurf(x1.T,x2.T,mv[0].T)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

RREV_trigger / landing_rate vs. impact_tdelta r2 =  0.5187896515856254
RREV_trigger vs. impact_tdelta r2 = 0.5128471788175983
landing_rate vs. impact_tdelta r2 = 0.001282295924067589


In [72]:
## AVG LANDING RATE PLOT
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=4)

pcm = ax.scatter(df['RREV_trigger'],df['vz_d'],df['landing_rate'],c=df['vz_d'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Vz_d')



ax.set_xlabel('RREV_trigger')
ax.set_ylabel('vz_d')
ax.set_zlabel('LR')

x1name = 'RREV_trigger'
x2name = 'vz_d'
yname = 'landing_rate'

x1,x2,y,mv,var1,var2 = linreg2(df[x1name],df[x2name],df[yname])
print(x1name+" / " + x2name + " vs. " + yname+ " r2 = ", mv[3])
print(x1name + " vs. " + yname + " r2 =" , var1[3])
print(x2name + " vs. " + yname + " r2 =" , var2[3])

ax.plot_trisurf(x1.T,x2.T,mv[0].T)


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

RREV_trigger / vz_d vs. landing_rate r2 =  0.051637377004846496
RREV_trigger vs. landing_rate r2 = 0.001537417738224356
vz_d vs. landing_rate r2 = 0.014689049602524817


In [74]:
## AVG LANDING RATE PLOT
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=4,vmax=5)

pcm = ax.scatter(df['vx_d'],df['vz_d'],df['My_d'],c=df['RREV_trigger'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='RREV')



ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('My_d')

x1name = 'vx_d'
x2name = 'vz_d'
yname = 'My_d'

x1,x2,y,mv,var1,var2 = linreg2(tmp_df[x1name],tmp_df[x2name],tmp_df[yname])
print(x1name+" / " + x2name + " vs. " + yname+ " r2 = ", mv[3])
print(x1name + " vs. " + yname + " r2 =" , var1[3])
print(x2name + " vs. " + yname + " r2 =" , var2[3])

ax.plot_trisurf(x1.T,x2.T,mv[0].T)

plt.show()
## I have more data for vz = (1.5,2.75) than vz = (3.00,4.0) so My_d differences are likely not a trend, just sparse data



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

vx_d / vz_d vs. My_d r2 =  0.13071752152629057
vx_d vs. My_d r2 = 0.029159741369487846
vz_d vs. My_d r2 = 0.09464898543944122


In [76]:

## AVG LANDING RATE PLOT
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
pcm = ax.scatter(df['vx_d'],df['vz_d'],-df['impact_eul'],c=df['landing_rate'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing %')

ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('Impact Angle')

x1name = 'vx_d'
x2name = 'vz_d'
yname = 'impact_eul'

x1,x2,y,mv,var1,var2 = linreg2(tmp_df[x1name],tmp_df[x2name],-tmp_df[yname])
print(x1name+" / " + x2name + " vs. " + yname+ " r2 = ", mv[3])
print(x1name + " vs. " + yname + " r2 =" , var1[3])
print(x2name + " vs. " + yname + " r2 =" , var2[3])

ax.plot_trisurf(x1.T,x2.T,mv[0].T)

# not sure if I should continue or which ones lin reg should be done for? ie avg or raw?

ax.set_zlim(0,180)
# ax.set_zticks([0,60,120,180])

## Failed landings in corner
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

vx_d / vz_d vs. impact_eul r2 =  0.1519321592575923
vx_d vs. impact_eul r2 = 0.04600337364517293
vz_d vs. impact_eul r2 = 0.11422069795244749


In [ ]:
## AVG LANDING RATE PLOT
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=0,vmax=180)
ax.plot_trisurf(df_avg['vx_d'],df_avg['vz_d'],-df_avg['impact_eul','mean'],cmap=cmap)


ax.set_xlim(0,2.75)
ax.set_ylim(1.0,4.0)
ax.set_zlim(-70,180)
ax.set_zticks([0,60,120,180])


ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('Impact Angle')

ax.set_title('Avg Impact Angle vs IC')
plt.show()

In [ ]:
## AVG LANDING RATE PLOT
fig = plt.figure()
ax = fig.add_subplot(111)


cmap = mpl.cm.Greys
norm = mpl.colors.Normalize(vmin=0,vmax=1)

ax.scatter(df['RREV_threshold'],df['My_d'],c=df['landing_rate'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing %')

ax.grid()
ax.set_xlim(3,7)
ax.set_ylim(0,10)
ax.hlines(7.77,3,7)
ax.text(3.05,8.0,'Motors Maxed Out')

ax.set_xlabel('RREV_thr')
ax.set_ylabel('My_d [N*mm]')
ax.set_title('Policy: My_d vs RREV_thr')

plt.show()

In [ ]:
## AVG LANDING RATE PLOT
fig = plt.figure()
ax = fig.add_subplot(111)


cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=0,vmax=3)

ax.scatter(df['vx_d'],df['My_d'],c=df['landing_rate'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing rate')

ax.grid()
ax.set_ylim(0,10)
# ax.set_xlim(3,7)

ax.hlines(7.77,3,7)
ax.text(3.05,-8.0,'Motors Maxed Out')

ax.set_xlabel('Vx_d')
ax.set_ylabel('My_d [N*mm]')
ax.set_title('Policy: My_d vs RREV_thr')
## As Vz increases RREV_trigger decreases to preserve time to contact. It also finds a host of My_d that work and they vary because of the robustness of the legs and the fact that landing success isn't deterministic.
plt.show()

In [ ]:
## AVG LANDING RATE PLOT
fig = plt.figure()
ax = fig.add_subplot(111)


cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=0,vmax=3)

ax.scatter(df['RREV_threshold'],df['My_d'],c=df['vz_d'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Vz_d')

ax.grid()
ax.set_ylim(0,10)
ax.set_xlim(3,7)

ax.hlines(7.77,3,7)
ax.text(3.05,-8.0,'Motors Maxed Out')

ax.set_xlabel('RREV_thr')
ax.set_ylabel('My_d [N*mm]')
ax.set_title('Policy: My_d vs RREV_thr')
## As Vz increases RREV_trigger decreases to preserve time to contact. It also finds a host of My_d that work and they vary because of the robustness of the legs and the fact that landing success isn't deterministic.
plt.show()

In [ ]:
## AVG LANDING RATE PLOT
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)

temp_df = df.query(f"vx_d=={0.0}").iloc[:][['vz_d','vx_d','RREV_threshold','My_d','landing_rate']]

pcm = ax.scatter(temp_df['RREV_threshold'],temp_df['vx_d'],temp_df['My_d'],c=temp_df['landing_rate'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='LR')

ax.set_zlim(0,10)
# ax.set_xlim(3,7)

ax.set_xlabel('RREV')
ax.set_ylabel('vx_d')
ax.set_zlabel('My_d')


plt.show()

In [ ]:
## AVG LANDING RATE PLOT
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)

pcm = ax.scatter(df['RREV_threshold'],df['landing_rate'],df['My_d'],c=df['landing_rate'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing %')

ax.set_zlim(0,10)
ax.set_xlim(3,7)

ax.set_xlabel('RREV_trigger')
ax.set_ylabel('Landing_Rate')
ax.set_zlabel('My_d')


plt.show()

In [ ]:
## AVG LANDING RATE PLOT
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)

pcm = ax.scatter(df['RREV_trigger'],df['vz_d'],df['My_d'],c=df['landing_rate'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing %')

ax.set_zlim(0,10)
ax.set_xlim(3,7)

ax.set_xlabel('RREV_trigger')
ax.set_ylabel('Vz_d')
ax.set_zlabel('My_d')


plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)

pcm = ax.scatter(df['vx_d'],df['My_d'],df['RREV_threshold'],c=df['landing_rate'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing %')

# ax.set_xlim(3,7)
# ax.set_ylim(0,-10)
# ax.set_zlim(0,-10)

ax.set_xlabel('Vz_d')
ax.set_ylabel('OF_y_trigger')
ax.set_zlabel('RREV_threshold')


plt.show()

In [ ]:
## AVG LANDING RATE PLOT
fig = plt.figure()
ax = fig.add_subplot(111)

cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=0,vmax=3)

pcm = ax.scatter(df_avg['RREV_threshold','mean'],df_avg['RREV_trigger','mean'],c=df_avg['vz_d'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Vz_d')

ax.grid()
ax.set_xlim(3,8)
ax.set_ylim(3,8)

ax.set_xlabel('RREV_threshold')
ax.set_ylabel('RREV_trigger')



plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)

pcm = ax.scatter(df['RREV_threshold'],df['OF_y'],df['My_d'],c=df['landing_rate'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing Rate')

ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)

ax.set_xlabel('RREV')
ax.set_ylabel('OF_y')
ax.set_zlabel('My_d')


plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)

pcm = ax.scatter(df['G1'],df['OF_y'],df['My_d'],c=df['landing_rate'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing Rate')

# ax.set_xlim(3,7)
# ax.set_ylim(0,-10)
ax.set_zlim(0,10)

ax.set_xlabel('G1')
ax.set_ylabel('OF_y')
ax.set_zlabel('My_d')


plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)

pcm = ax.scatter(df['G1'],df['G2'],df['My_d'],c=df['landing_rate'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing Rate')

# ax.set_xlim(3,7)
# ax.set_ylim(0,-10)
ax.set_zlim(0,10)

ax.set_xlabel('G1')
ax.set_ylabel('G2')
ax.set_zlabel('My_d')


plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)

pcm = ax.scatter(df['G1'],df['vx_d'],df['My_d'],c=df['landing_rate'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing Rate')

# ax.set_xlim(3,7)
# ax.set_ylim(0,-10)
ax.set_zlim(0,10)

ax.set_xlabel('G1')
ax.set_ylabel('vx_d')
ax.set_zlabel('My_d')


plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")




cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing Rate')
ax.scatter(df['RREV_threshold'],df['OF_y'],df['My_d'],c=df['landing_rate'],cmap=cmap,norm=norm)




ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)

ax.set_xlabel('RREV')
ax.set_ylabel('OF_y')
ax.set_zlabel('My_d')


plt.show()

In [ ]:
from sklearn.linear_model import LinearRegression


df_reg = df.query(f"landing_rate>={0.8} & RREV_threshold>{3.5}")


X = df_reg[['RREV_threshold','OF_y']]
Y = df_reg['My_d']

reg = LinearRegression(normalize=True)
reg.fit(X,Y)

intercept = reg.intercept_
coeffs = reg.coef_

print(intercept)
print(coeffs)


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")


My_reg = intercept + coeffs[0]*df_reg['RREV_threshold'] + coeffs[1]*df_reg['OF_y']
ax.plot_trisurf(df_reg['RREV_threshold'],df_reg['OF_y'],My_reg)


cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing Rate')
ax.scatter(df_reg['RREV_threshold'],df_reg['OF_y'],df_reg['My_d'],c=df_reg['landing_rate'],cmap=cmap,norm=norm)




ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)

ax.set_xlabel('RREV')
ax.set_ylabel('OF_y')
ax.set_zlabel('My_d')


plt.show()

In [ ]:
def r_squared(y, y_hat):
    y_bar = y.mean()
    ss_tot = ((y-y_bar)**2).sum()
    ss_res = ((y-y_hat)**2).sum()
    return 1 - (ss_res/ss_tot)

R2 = r_squared(df_reg['My_d'],My_reg)
print(f"R^2: {R2:.3f}")

In [ ]:
print(reg.score(X,Y))

In [ ]:

# ignore this , just use linreg2
ypred,intercept,coef,R2 = linreg(X,Y)

fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")
ax.plot_trisurf(X.iloc[:, 0].values,X.iloc[:, 1].values,ypred)
cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing Rate')
ax.scatter(df_reg['RREV_threshold'],df_reg['OF_y'],df_reg['My_d'],c=df_reg['landing_rate'],cmap=cmap,norm=norm)


In [ ]:
print(X.iloc[:, 0].values)
